In [3]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
loader = UnstructuredPDFLoader("../Downloads/Introduction to Genetics.pdf")

In [11]:
data = loader.load()

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 618.81it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 267.80it/s]
0it [00:00, ?it/s]
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 3711.77it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 280.89it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 428.65it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [28]:
print (f'There are {len(data)} document(s)')
print (f'There are {len(data[0].page_content)} characters in the document')

There are 1 document(s)
There are 511642 characters in the document


In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [30]:
print (f'There are {len(texts)} documents')

There are 702 documents


In [15]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY =os.getenv.('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv.('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv.('PINECONE_API_ENV')

In [17]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [21]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV,
)
index_name = "test"

In [22]:
search = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)


In [23]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [24]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [25]:
query = "What is RNA?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [26]:
chain.run(input_documents=docs, question=query)

' RNA (ribonucleic acid) is a type of nucleic acid that is used as the genetic material in some viruses. It is similar to DNA, but is single-stranded and contains the sugar ribose instead of deoxyribose. RNA is involved in the transcription of genetic information from DNA to proteins.'